In [ ]:
# ensure that any code changes are immediately reflected
%reload_ext autoreload
%autoreload 2

In [ ]:
from mostlyai import MostlyAI
import pandas as pd
import os
os.environ['MOSTLY_AI_PASSWORD'] = 'Mostly01!'
client = MostlyAI(
    # api_key = 'xxx',         # or MOSTLYAI_API_KEY env var; this is bound to a single user account
    # base_url = 'https://..'  # optional; defaults to our free version
)

## CENSUS

In [ ]:
census = pd.read_csv('https://github.com/mostly-ai/public-demo-data/raw/dev/census/census.csv.gz').sample(n=1_000)
g = client.generators.create(
    name='census',
    description='census',
    tables=[{'name': 'census', 'data': census}],
)

In [ ]:
sd = client.synthetic_datasets.create(generatorId=g.id)

In [ ]:
dfs = sd.data()
dfs['census']

In [ ]:
#seed = pd.DataFrame({'sex': ['Male', 'Female'] * 100, 'age': [49] * 200})
#sd = client.synthetic_datasets.create(generatorId=g.id, tables=[{'name': 'users', 'sampleSeedData': seed}])

## CDNOW

In [ ]:
# CDNOW from HTTP
g = client.generators.create(
    name='CDNOW',
    tables=[
        {'name': 'users', 'data': 'https://github.com/mostly-ai/public-demo-data/raw/dev/cdnow/users.csv.gz', 'primaryKey': 'id'}, 
        {'name': 'purchases', 'data': 'https://github.com/mostly-ai/public-demo-data/raw/dev/cdnow/purchases.csv.gz', 
         'foreignKeys': [{'column': 'users_id', 'referencedTable': 'users', 'isContext': True}]}
    ]
)

In [ ]:
acc = g.tables[0].model.metrics.accuracy.univariate
print(f"accuracy: {acc:.2%}")

In [ ]:
sd = client.synthetic_datasets.create(generatorId=g.id, tables=[{'name': 'users', 'sampleSeedData': seed}])

In [ ]:
dfs = sd.data()
dfs

# clone a generator

In [ ]:
g = client.generators.get('cafd2ce2-1cc3-41b7-85a5-ed12a469d08b')
print(f"{str(g.id)=}, {g.name=}, {len(g.tables)=}, {g.training_status.value=}")

In [ ]:
# clone a generator, and re-provide the data
config = g.to_dict()
config['tables'][0]['data'] = 'https://github.com/mostly-ai/public-demo-data/raw/dev/census/census.csv.gz'
g2 = client.generators.create(**config, start=False)

In [ ]:
print(f"{str(g2.id)=}, {g2.name=}, {len(g2.tables)=}, {g2.training_status.value=}")

In [ ]:
g2.training.start()
g2 = g2.training.wait(interval=5)
print(f"{g2.training_status.value=}")

## list generators

In [ ]:
for g in client.generators.list():
    print(f"name:{g.name}, status:{g.training_status.value}, accuracy:{g.accuracy}")